In [3]:
pip install langchain langchain-community chromadb sentence-transformers openai langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.2 MB/s eta 0:00:00


In [4]:
# rag_langchain_chroma_vllm.py
# pip install langchain langchain-community chromadb sentence-transformers openai langchain_openai

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_openai import ChatOpenAI  # буде працювати з vLLM-сервером

# -------------------------------------------------------
# 1) Вихідні дані для індексації
# -------------------------------------------------------
# У реальному кейсі тут підвантажуються PDF/DOCX/HTML файли,
# але для демонстрації беремо кілька простих рядків.
raw_docs = [
    "RAG (Retrieval-Augmented Generation) поєднує пошук у зовнішніх джерелах знань з LLM.",
    "Chroma та FAISS — це популярні векторні сховища для семантичного пошуку.",
    "LangChain і LlamaIndex надають інструменти для створення RAG-пайплайнів.",
    "SentenceTransformers all-MiniLM-L6-v2 — легка модель для побудови ембеддингів.",
]

# -------------------------------------------------------
# 2) Попередня обробка: розбиття текстів на фрагменти
# -------------------------------------------------------
# Якщо документи довгі, їх треба розбити на шматки,
# щоб ембеддинги були якісними і релевантними.
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
docs = splitter.create_documents(raw_docs)

# -------------------------------------------------------
# 3) Побудова ембеддингів та векторної бази Chroma
# -------------------------------------------------------
# HuggingFaceEmbeddings — безкоштовна альтернатива для семантичних ембеддингів.
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chroma — легка векторна база з можливістю персисту на диск.
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=emb,
    persist_directory="chroma_db"  # каталог для збереження індекса
)

# retriever = інтерфейс, який повертає k найближчих фрагментів до запиту
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# -------------------------------------------------------
# 4) LLM через vLLM (OpenAI-сумісний API)
# -------------------------------------------------------
# Перед цим треба запустити vLLM сервер (див. інструкцію в docs.vllm.ai).
# Наприклад: http://localhost:8000/v1
llm = ChatOpenAI(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # або інша завантажена модель
    base_url="http://localhost:8000/v1",         # вказуємо адресу vLLM сервера
    api_key="EMPTY"                              # vLLM не вимагає реального ключа
)

# -------------------------------------------------------
# 5) Промпт-шаблон для генерації з урахуванням контексту
# -------------------------------------------------------
# У {context} підставляться знайдені релевантні документи,
# у {question} — запит користувача.
prompt = ChatPromptTemplate.from_template(
    "Використай наведений контекст, щоб відповісти стисло і по суті.\n"
    "Контекст:\n{context}\n\nПитання: {question}"
)

# -------------------------------------------------------
# 6) RAG-ланцюг: retriever -> prompt -> LLM -> відповідь
# -------------------------------------------------------
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# -------------------------------------------------------
# 7) Виконання запиту користувача
# -------------------------------------------------------
question = "Що таке RAG і які інструменти можна використати для індексації?"
answer = rag_chain.invoke(question)

print("Q:", question)
print("A:", answer)

# -------------------------------------------------------
# 8) При потребі можна додати нові документи в базу:
# -------------------------------------------------------
# new_docs = splitter.create_documents(["Додатковий текст"])
# vectordb.add_documents(new_docs)
# vectordb.persist()

/tmp/ipython-input-241135199.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dataset

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

APIConnectionError: Connection error.